In [3]:
import os
import re
import zipfile
import subprocess
import pandas as pd
import numpy as np
import zipfile as zf
from datetime import datetime

In [4]:
DATASET = 'ml-1m'  # only support "ml-100k" and "ml-1m" now
RAW_PATH = os.path.join('./', DATASET)

RANDOM_SEED = 0
NEG_ITEMS = 99

# Load Data

1. Load interaction data and item metadata
2. Filter out items with less than 5 interactions
3. Calculate basic statistics

In [5]:
# download data if not exists

if not os.path.exists(RAW_PATH):
    subprocess.call('mkdir ' + RAW_PATH, shell=True)
if not os.path.exists(os.path.join(RAW_PATH, DATASET + '.zip')):
    print('Downloading data into ' + RAW_PATH)
    subprocess.call(
        'cd {} && curl -O http://files.grouplens.org/datasets/movielens/{}.zip'
        .format(RAW_PATH, DATASET), shell=True)

In [6]:
with zipfile.ZipFile(os.path.join(RAW_PATH, DATASET + '.zip')) as z:
    if DATASET == 'ml-100k':
        with z.open(os.path.join(DATASET, 'u.data')) as f:
            data_df = pd.read_csv(f, sep="\t", header=None)
        with z.open(os.path.join(DATASET, 'u.item')) as f:
            meta_df = pd.read_csv(f, sep='|', header=None, encoding='ISO-8859-1')
    elif DATASET == 'ml-1m':
        with z.open(os.path.join(DATASET, 'ratings.dat')) as f:
            data_df = pd.read_csv(f, sep='::', header=None , encoding='ISO-8859-1', engine='python')
        with z.open(os.path.join(DATASET, 'movies.dat')) as f:
            meta_df = pd.read_csv(f, sep='::', header=None , encoding='ISO-8859-1', engine='python')

In [7]:
data_df.columns = ['user_id', 'item_id', 'label', 'time']
data_df.head()

,user_id,item_id,label,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [9]:
genres = [
    'i_Action', 'i_Adventure', 'i_Animation', "i_Children's", 'i_Comedy', 'i_Crime', 
    'i_Documentary', 'i_Drama', 'i_Fantasy', 'i_Film-Noir', 'i_Horror', 'i_Musical', 
    'i_Mystery', 'i_Romance', 'i_Sci-Fi', 'i_Thriller', 'i_War', 'i_Western', 'i_Other'
]

if DATASET == 'ml-100k':
    item_df = meta_df.drop([1, 3, 4], axis=1)
    item_df.columns = ['item_id', 'i_year'] + genres
elif DATASET == 'ml-1m':
    item_df = meta_df.copy()
    item_df.columns = ['item_id', 'title', 'genre']
    genre_dict = dict()
    for g in genres:
        genre_dict[g] = np.zeros(len(item_df), dtype=np.int32)
    item_genre = item_df['genre'].apply(lambda x: x.split('|')).values
    for idx, genre_lst in enumerate(item_genre):
        for g in genre_lst:
            genre_dict['i_' + g][idx] = 1
    for g in genres:
        item_df[g] = genre_dict[g]
    item_df = item_df.drop(columns=['title', 'genre'])

item_df.head()


,item_id,i_Action,i_Adventure,i_Animation,i_Children's,i_Comedy,i_Crime,i_Documentary,i_Drama,i_Fantasy,i_Film-Noir,i_Horror,i_Musical,i_Mystery,i_Romance,i_Sci-Fi,i_Thriller,i_War,i_Western,i_Other
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
# Only retain users and items with at least 5 associated interactions

# print('Filter before:', len(data_df))
# filter_before = -1
# while filter_before != len(data_df):
#     filter_before = len(data_df)
#     for stage in ['user_id', 'item_id']:
#         val_cnt = data_df[stage].value_counts()
#         cnt_df = pd.DataFrame({stage: val_cnt.index, 'cnt': val_cnt.values})
#         data_df = pd.merge(data_df, cnt_df, on=stage, how='left')
#         data_df = data_df[data_df['cnt'] >= 5].drop(columns=['cnt'])
# print('Filter after:', len(data_df))

item_df = item_df[item_df['item_id'].isin(data_df['item_id'])]  # remove unuseful metadata

### Statistics

In [62]:
n_users = data_df['user_id'].value_counts().size
n_items = data_df['item_id'].value_counts().size
n_clicks = len(data_df)
min_time = data_df['time'].min()
max_time = data_df['time'].max()

In [63]:
time_format = '%Y-%m-%d'

print('# Users:', n_users)
print('# Items:', n_items)
print('# Interactions:', n_clicks)
print('Time Span: {}/{}'.format(
    datetime.utcfromtimestamp(min_time).strftime(time_format),
    datetime.utcfromtimestamp(max_time).strftime(time_format))
)

# Users: 6040
# Items: 3700
# Interactions: 982089
Time Span: 2000-04-25/2003-02-28


# Build Dataset

### Interaction data

In [64]:
np.random.seed(RANDOM_SEED)

In [65]:
out_df = data_df[['user_id', 'item_id', 'time']]
out_df = out_df.drop_duplicates(['user_id', 'item_id', 'time'])
out_df.sort_values(by=['time', 'user_id'], kind='mergesort', inplace=True)
out_df = out_df.reset_index(drop=True)
out_df.head()

,user_id,item_id,time
0,6040,580,956703954
1,6040,2192,956703954
2,6040,1782,956703977
3,6040,1840,956703977
4,6040,560,956704056


In [66]:
# reindex (start from 1)

uids = sorted(out_df['user_id'].unique())
user2id = dict(zip(uids, range(1, len(uids) + 1)))
iids = sorted(out_df['item_id'].unique())
item2id = dict(zip(iids, range(1, len(iids) + 1)))

out_df['user_id'] = out_df['user_id'].apply(lambda x: user2id[x])
out_df['item_id'] = out_df['item_id'].apply(lambda x: item2id[x])
out_df.head()

,user_id,item_id,time
0,6040,580,956703954
1,6040,2189,956703954
2,6040,1780,956703977
3,6040,1838,956703977
4,6040,560,956704056


In [67]:
# leave one out spliting

clicked_item_set = dict()
for user_id, seq_df in out_df.groupby('user_id'):
    clicked_item_set[user_id] = set(seq_df['item_id'].values.tolist())
    
def generate_dev_test(data_df):
    result_dfs = []
    n_items = data_df['item_id'].value_counts().size
    for idx in range(2):
        result_df = data_df.groupby('user_id').tail(1).copy()
        data_df = data_df.drop(result_df.index)
        neg_items = np.random.randint(1, n_items + 1, (len(result_df), NEG_ITEMS))
        for i, uid in enumerate(result_df['user_id'].values):
            user_clicked = clicked_item_set[uid]
            for j in range(len(neg_items[i])):
                while neg_items[i][j] in user_clicked:
                    neg_items[i][j] = np.random.randint(1, n_items + 1)
        result_df['neg_items'] = neg_items.tolist()
        result_dfs.append(result_df)
    return result_dfs, data_df

In [68]:
leave_df = out_df.groupby('user_id').head(1)
data_df = out_df.drop(leave_df.index)

[test_df, dev_df], data_df = generate_dev_test(data_df)
train_df = pd.concat([leave_df, data_df]).sort_index()

len(train_df), len(dev_df), len(test_df)

(970009, 6040, 6040)

In [69]:
train_df.head()

,user_id,item_id,time
0,6040,580,956703954
1,6040,2189,956703954
2,6040,1780,956703977
3,6040,1838,956703977
4,6040,560,956704056


In [70]:
test_df.head()

,user_id,item_id,time,neg_items
747,6034,882,956712388,"[2733, 2608, 1654, 3265, 836, 764, 1732, 3432,..."
956,6035,815,956713620,"[3013, 1072, 640, 2037, 1156, 973, 2917, 1205,..."
1014,6033,1407,956714112,"[2345, 1353, 3092, 608, 1435, 2499, 1273, 181,..."
1029,6038,1317,956714815,"[2122, 298, 2968, 731, 1971, 2925, 1796, 1824,..."
1494,6031,1278,956718549,"[3609, 2677, 1216, 1220, 1332, 137, 953, 860, ..."


In [71]:
# save results

train_df.to_csv(os.path.join(RAW_PATH, 'train.csv'), sep='\t', index=False)
dev_df.to_csv(os.path.join(RAW_PATH, 'dev.csv'), sep='\t', index=False)
test_df.to_csv(os.path.join(RAW_PATH, 'test.csv'), sep='\t', index=False)

### Item Metadata

In [72]:
print(item_df.columns)
item_df['item_id'] = item_df['item_id'].apply(lambda x: item2id[x])

if DATASET == 'ml-1m':
    item_df['i_year'] = item_df['title'].apply(lambda x: int(re.match('.+\((\d{4})\)$', x).group(1)))
    item_df = item_df.drop(columns=['title'])
elif DATASET == 'ml-100k':
    item_df['i_year'] = item_df['i_year'].apply(lambda x: int(str(x).split('-')[-1]) if pd.notnull(x) else 0)
seps = [1900, 1940, 1950, 1960, 1970, 1980, 1985] + list(range(1990, int(item_df['i_year'].max() + 2)))
year_dict = {}
for i, sep in enumerate(seps[:-1]):
    for j in range(seps[i], seps[i + 1]):
        year_dict[j] = i + 1
item_df['i_year'] = item_df['i_year'].apply(lambda x: year_dict[x] if x > 0 else 0)
    
item_df.head()

Index([0, 1, 2], dtype='int64')


KeyError: 'item_id'

In [166]:
# save results

item_df.to_csv(os.path.join(RAW_PATH, 'item_meta.csv'), sep='\t', index=False)